In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
gender=pd.read_csv('../input/gender_submission.csv')
test.info()

['test.csv', 'train.csv', 'gender_submission.csv']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [2]:
train=train.drop(['Name','Ticket','Cabin'],axis=1)
train["Embarked_C"] = train["Embarked"] == "C"
train["Embarked_S"] = train["Embarked"] == "S"
train["Embarked_Q"] = train["Embarked"] == "Q"

train["male"] = train["Sex"] == "male"
train["female"] = train["Sex"] == "female"
train=train.drop(['Embarked','Sex'],axis=1)
train=train.dropna(axis=0)
train.head()
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    714 non-null int64
Survived       714 non-null int64
Pclass         714 non-null int64
Age            714 non-null float64
SibSp          714 non-null int64
Parch          714 non-null int64
Fare           714 non-null float64
Embarked_C     714 non-null bool
Embarked_S     714 non-null bool
Embarked_Q     714 non-null bool
male           714 non-null bool
female         714 non-null bool
dtypes: bool(5), float64(2), int64(5)
memory usage: 48.1 KB


In [3]:
test['TT']=test['Name'].str.extract(' ([A-Za-z]+)\.',expand=False)
test=test.drop(['Name'],axis=1)
test["Embarked_C"] = test["Embarked"] == "C"
test["Embarked_S"] = test["Embarked"] == "S"
test["Embarked_Q"] = test["Embarked"] == "Q"

test["male"] = test["Sex"] == "male"
test["female"] = test["Sex"] == "female"
test=test.drop(['Embarked','Sex'],axis=1)
test=test.drop(['Cabin','Ticket'],axis=1)

TT_mapping={"Mr":0,"Miss":1,"Mrs":2,"Master":3,"Dr":0,"Rev":3,"Col:":3,"Major":3,"Mlle":3,"Countess":3,"Ms":3,"Lady":1,"Jonkheer":3,"Don":3,"Mme":1,"Capt":0,"Sir":0}
test["TT"]=test["TT"].map(TT_mapping)
test["TT"]=test["TT"].fillna(3)
test["TT"].astype(int)
test.isnull().sum()

PassengerId     0
Pclass          0
Age            86
SibSp           0
Parch           0
Fare            1
TT              0
Embarked_C      0
Embarked_S      0
Embarked_Q      0
male            0
female          0
dtype: int64

In [4]:
test['Age'].fillna(test.groupby('TT')["Age"].transform('median'),inplace=True)
test['Fare'].fillna(test.groupby('TT')['Fare'].transform('median'),inplace=True)
test=test.drop(["TT"],axis=1)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked_C     418 non-null bool
Embarked_S     418 non-null bool
Embarked_Q     418 non-null bool
male           418 non-null bool
female         418 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 21.7 KB


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test=train_test_split(train[['Pclass','Age','SibSp','Parch','Fare','Embarked_C','Embarked_S','Embarked_Q','male','female']],train[['Survived']],test_size=0.1,random_state=7)


In [6]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(max_depth = 5, random_state = 7)
tree_clf.fit(X_train, y_train)
print('Score: {}'.format(tree_clf.score(X_train, y_train)))

Score: 0.8613707165109035


In [7]:
test_one=test.drop(["PassengerId"],axis=1)
prediction=tree_clf.predict(test_one)
submission=pd.DataFrame({
    "PassengerId":test['PassengerId']
})
submission["Survived"]=prediction
submission.to_csv('submission.csv',index=False)